# **Stock Prices Predictions with Machine Learning**

This Notebook will contain the modeling phases needed to predict stock prices using a deep learning model.
The stocks analyzed will be the following:
* IBM
* AAPL (Apple Inc.)
* AMZN (Amazon Inc.)
* GOOGL (Alphabet Inc.)


# Data preparation

Data must be prepared in order to be processed by DeepAR model:
* Train/test set split
* Save Data locally
* Upload to S3

## Save Data Locally

In [161]:
data_dir = 'stock_deepar'

In [162]:
import os

In [163]:
data_dir_csv = os.path.join(data_dir, 'csv') # The folder we will use for storing data
if not os.path.exists(data_dir_csv): # Make sure that the folder exists
    os.makedirs(data_dir_csv)

In [164]:
interval ='D'

In [165]:
# IBM
df_ibm_train.to_csv(os.path.join(data_dir_csv, 'ibm_train.csv'), header=True, index=True)
df_ibm_test.to_csv(os.path.join(data_dir_csv, 'ibm_test.csv'), header=True, index=True)
df_ibm_valid.to_csv(os.path.join(data_dir_csv, 'ibm_valid.csv'), header=True, index=True)

In [166]:
# Apple Inc.
df_aapl_train.to_csv(os.path.join(data_dir_csv, 'aapl_train.csv'), header=True, index=True)
df_aapl_test.to_csv(os.path.join(data_dir_csv, 'aapl_test.csv'), header=True, index=True)
df_aapl_valid.to_csv(os.path.join(data_dir_csv, 'aapl_valid.csv'), header=True, index=True)

In [167]:
# Amazon.com
df_amzn_train.to_csv(os.path.join(data_dir_csv, 'amzn_train.csv'), header=True, index=True)
df_amzn_test.to_csv(os.path.join(data_dir_csv, 'amzn_test.csv'), header=True, index=True)
df_amzn_valid.to_csv(os.path.join(data_dir_csv, 'amzn_valid.csv'), header=True, index=True)

In [168]:
# Alphabet Inc.
df_googl_train.to_csv(os.path.join(data_dir_csv, 'googl_train.csv'), header=True, index=True)
df_googl_test.to_csv(os.path.join(data_dir_csv, 'googl_test.csv'), header=True, index=True)
df_googl_valid.to_csv(os.path.join(data_dir_csv, 'googl_valid.csv'), header=True, index=True)

### JSON serialization

In order to feed DeepAR model, JSON files must be prepared from data.
I'll dispose two kind of JSON inputs:
* one with "dynamic features", to use a DeepAR API terminology: all dataset features except for target column and related one ('Adj Close', 'Close');
* one without "dynamic features: only 'Adj Close' column will be fed to DeepAR model.

### JSON files

Now I'm going to convert data to JSON file format, in order to feed the DeepAR model correctly

As already announced, I will create two kind of time series, one with a list of dynamic features `dyn_feat`and the other one with only the target column (`Adj Close`) time series. 

In [169]:
# initializing train/test dataframe lists to iterate on them
dfs_train = [df_ibm_train, df_aapl_train, df_amzn_train, df_googl_train]
dfs_test = [df_ibm_test, df_aapl_test, df_amzn_test, df_googl_test]

Creating local storage path:

In [170]:
data_dir_json = os.path.join(data_dir, 'json')
if not os.path.exists(data_dir_json): # Make sure that the folder exists
    os.makedirs(data_dir_json)

Serializing data to json files

In [171]:
from source_deepar.deepar_utils import ts2json_serialize

Dataset with the `Adj Close` time series alone:

Training data:

In [172]:
data_dir_json_train = os.path.join(data_dir_json, 'train') # The folder we will use for storing data
if not os.path.exists(data_dir_json_train): # Make sure that the folder exists
    os.makedirs(data_dir_json_train)

In [173]:
for df, m in zip(dfs_train, mnemonics):
    ts2json_serialize(df, data_dir_json_train, m+'.json')

Test data:

In [174]:
data_dir_json_test = os.path.join(data_dir_json, 'test') # The folder we will use for storing data
if not os.path.exists(data_dir_json_test): # Make sure that the folder exists
    os.makedirs(data_dir_json_test)

In [175]:
for df, m in zip(dfs_test, mnemonics):
    ts2json_serialize(df, data_dir_json_test, m+'.json')

Dataset containing dynamic features:

Training data:

In [176]:
data_dir_json_dyn_feat = os.path.join(data_dir_json, 'w_dyn_feat')

In [177]:
data_dir_json_train_dyn_feat = os.path.join(data_dir_json_dyn_feat, 'train') # The folder we will use for storing data
if not os.path.exists(data_dir_json_train_dyn_feat): # Make sure that the folder exists
    os.makedirs(data_dir_json_train_dyn_feat)

In [178]:
for df, m in zip(dfs_train, mnemonics):
    ts2json_serialize(df, data_dir_json_train_dyn_feat, m+'.json', dyn_feat=['Open'])

Test data:

In [179]:
data_dir_json_test_dyn_feat = os.path.join(data_dir_json_dyn_feat, 'test') # The folder we will use for storing data
if not os.path.exists(data_dir_json_test_dyn_feat): # Make sure that the folder exists
    os.makedirs(data_dir_json_test_dyn_feat)

In [180]:
for df, m in zip(dfs_test, mnemonics):
    ts2json_serialize(df, data_dir_json_test_dyn_feat, m+'.json', dyn_feat=['Open'])

Validation data:

In [181]:
data_dir_json_test_dyn_feat = os.path.join(data_dir_json_dyn_feat, 'test') # The folder we will use for storing data
if not os.path.exists(data_dir_json_test_dyn_feat): # Make sure that the folder exists
    os.makedirs(data_dir_json_test_dyn_feat)

In [182]:
for df, m in zip(dfs_test, mnemonics):
    ts2json_serialize(df, data_dir_json_test_dyn_feat, m+'.json', dyn_feat=['Open'])

## Uploading data to S3

In [183]:
import sagemaker
# Define IAM role and session
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

#Define training data location
s3_data_key = 'stock_deepar/train_artifacts'
s3_bucket = sagemaker_session.default_bucket()
interval = 'D' #Use D or H
s3_output_path = "s3://{}/{}/{}/output".format(s3_bucket, s3_data_key, interval)


In [184]:
# *unique* train/test prefixes
train_prefix   = '{}/{}'.format(data_dir_json, 'train')
test_prefix    = '{}/{}'.format(data_dir_json, 'test')
train_prefix_dyn_feat   = '{}/{}'.format(data_dir_json_dyn_feat, 'train')
test_prefix_dyn_feat    = '{}/{}'.format(data_dir_json_dyn_feat, 'test')

In [185]:
input_data_train = sagemaker_session.upload_data(path=data_dir_json_train, bucket=s3_bucket, key_prefix=train_prefix)

In [186]:
input_data_test = sagemaker_session.upload_data(path=data_dir_json_test, bucket=s3_bucket, key_prefix=test_prefix)

In [187]:
input_data_train_dyn_feat = sagemaker_session.upload_data(path=data_dir_json_train, bucket=s3_bucket, key_prefix=train_prefix_dyn_feat)

In [188]:
input_data_test_dyn_feat = sagemaker_session.upload_data(path=data_dir_json_test, bucket=s3_bucket, key_prefix=test_prefix_dyn_feat)

### Set DeepAR specific hyperparameters

In [185]:
from source_deepar import deepar_utils

# setting target columns
target_column = 'Adj Close'
    
    
hyperparameters = {
    "prediction_length": str(prediction_length[1]), #number of time-steps model is trained to predict, always generates forecasts with this length
    "context_length": str(context_length[1]), #number of time-points that the model gets to see before making the prediction, should be about same as the prediction_length
    "time_freq": interval, #granularity of the time series in the dataset
    "epochs": "200", #maximum number of passes over the training data
    "early_stopping_patience": "40", #training stops when no progress is made within the specified number of epochs
    "num_layers": "2", #number of hidden layers in the RNN, typically range from 1 to 4    
    "num_cells": "40", #number of cells to use in each hidden layer of the RNN, typically range from 30 to 100
    "mini_batch_size": "128", #size of mini-batches used during training, typically values range from 32 to 512
    "learning_rate": "1e-3", #learning rate used in training. Typical values range from 1e-4 to 1e-1
    "dropout_rate": "0.1", # dropout rate to use for regularization, typically less than 0.2. 
    "likelihood": "gaussian" #noise model used for uncertainty estimates - gaussian/beta/negative-binomial/student-T/deterministic-L1
}

In [186]:

# Define IAM role and session
role = sagemaker.get_execution_role()
session = sagemaker.Session()

#Obtain container image URI for SageMaker-DeepAR algorithm, based on region
region = session.boto_region_name
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")
print("Model will be trained using container image : {}".format(image_name))

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


Model will be trained using container image : 495149712605.dkr.ecr.eu-central-1.amazonaws.com/forecasting-deepar:1


## Estimator Instantiation

In [191]:
from sagemaker.estimator import Estimator

# instantiate a DeepAR estimator
estimator = Estimator(image_uri=image_name,
                      sagemaker_session=sagemaker_session,
                      image_name=image_name,
                      role=role,
                      train_instance_count=1,
                      train_instance_type='ml.c4.xlarge',
                      output_path=s3_output_path,
                      hyperparameters=hyperparameters
                      )

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Training Job Creation

Creation of a training job with stand alone time series (no dynamic features provided).

In [ ]:
%%time
# train and test channels
data_channels = {
    "train": input_data_train,
    "test": input_data_test
}

# fit the estimator
estimator.fit(inputs=data_channels)

## Deploy and Create a Predictor

Now that we have trained a model, we can use it to perform predictions by deploying it to a predictor endpoint.

Remember to **delete the endpoint** at the end of this notebook. A cell at the very bottom of this notebook will be provided, but it is always good to keep, front-of-mind.

In [185]:
endpoint_name = 'DeepAR-ml-spp'

In [ ]:
# create a predictor

from sagemaker.predictor import json_serializer, json_deserializer

In [ ]:
# run it once, then update the endpoint
%%time
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name=endpoint_name,
    #content_type="application/json" # specify that it will accept/produce JSON
    update_endpoint=True,
    serializer=json_serializer,
    deserializer=json_deserializer
)

In [203]:
%%time

# update an endpoint

predictor = estimator.update_endpoint(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
)

-----------------!CPU times: user 281 ms, sys: 20 ms, total: 301 ms
Wall time: 8min 32s


In [186]:
from source_deepar.deepar_utils import DeepARPredictor

In [189]:
json_predictor = DeepARPredictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)
json_predictor.set_prediction_parameters(interval, prediction_length[1])

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# Generating Predictions

According to the [inference format](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar-in-formats.html) for DeepAR, the `predictor` expects to see input data in a JSON format, with the following keys:
* **instances**: A list of JSON-formatted time series that should be forecast by the model.
* **configuration** (optional): A dictionary of configuration information for the type of response desired by the request.

Within configuration the following keys can be configured:
* **num_samples**: An integer specifying the number of samples that the model generates when making a probabilistic prediction.
* **output_types**: A list specifying the type of response. We'll ask for **quantiles**, which look at the list of num_samples generated by the model, and generate [quantile estimates](https://en.wikipedia.org/wiki/Quantile) for each time point based on these values.
* **quantiles**: A list that specified which quantiles estimates are generated and returned in the response.


Below is an example of what a JSON query to a DeepAR model endpoint might look like.

```
{
 "instances": [
  { "start": "2009-11-01 00:00:00", "target": [4.0, 10.0, 50.0, 100.0, 113.0] },
  { "start": "1999-01-30", "target": [2.0, 1.0] }
 ],
 "configuration": {
  "num_samples": 50,
  "output_types": ["quantiles"],
  "quantiles": ["0.5", "0.9"]
 }
}
```

## JSON Prediction Request

The code below accepts a **list** of time series as input and some configuration parameters. It then formats that series into a JSON instance and converts the input into an appropriately formatted JSON_input.

In [199]:
from source_deepar.deepar_utils import __series_to_json_obj
import json

In [243]:
def json_predictor_input(input_ts, target_column='Adj Close', dyn_feat=[], start=None, num_samples=50, quantiles=['0.1', '0.5', '0.9']):
    '''Accepts a list of input time series and produces a formatted input.
       :input_ts: An list of input time series.
       :num_samples: Number of samples to calculate metrics with.
       :quantiles: A list of quantiles to return in the predicted output.
       :return: The JSON-formatted input.
       '''
    # request data is made of JSON objects (instances)
    # and an output configuration that details the type of data/quantiles we want
    
    instances = []
    for k in range(len(input_ts)):
        # get JSON objects for input time series
        instances.append(__series_to_json_obj(input_ts[k], target_column=target_column, dyn_feat=dyn_feat, start=start))

    # specify the output quantiles and samples
    '''configuration = {"num_samples": num_samples, 
                     "output_types": ["quantiles"], 
                     "quantiles": quantiles}

    request_data = {"instances": instances, 
                    "configuration": configuration}
    
    '''
    
    configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles"],
            "quantiles": quantiles
    }

    request_data = {
            "instances": [instances],
            "configuration": configuration
    }
    
    json_request = json.dumps(request_data)
    
    return json_request

### Get a Prediction

We can then use this function to get a prediction for a formatted time series!

In the next cell, I'm getting an input time series and known target, and passing the formatted input into the predictor endpoint to get a resultant prediction.

In [ ]:
# get all input and target (test) time series
# input_ts = [df_ibm_train, df_aapl_train, df_amzn_train, df_googl_train]
# target_ts = [df_ibm_test, df_aapl_test, df_amzn_test, df_googl_test]

input_ts = [df_ibm_train]
target_ts = [df_ibm_test]

# get formatted input time series
json_input_ts = json_predictor_input(input_ts)

# get the prediction from the predictor
json_prediction = predictor.predict(json_input_ts)

#print(json_prediction)

### Predicting IBM Stock

In [190]:
# get all input and target (test) time series
# input_ts = [df_ibm_train, df_aapl_train, df_amzn_train, df_googl_train]
# target_ts = [df_ibm_test, df_aapl_test, df_amzn_test, df_googl_test]

input_ts = [df_ibm_train]
target_ts = [df_ibm_test]

# get the prediction from the predictor
json_prediction = json_predictor.predict(input_ts)



In [191]:
print(json_prediction)

[                   0.1         0.9         0.5
2021-01-07  121.835075  127.992470  125.059486
2021-01-08  122.355553  128.646957  124.954491
2021-01-09  119.964478  128.169891  124.520325
2021-01-10  121.023933  128.718262  124.628288
2021-01-11  119.816162  128.747482  124.592346
2021-01-12  120.824181  129.297180  125.081215
2021-01-13  119.372238  128.318588  124.544899
2021-01-14  119.224167  128.399811  123.815605
2021-01-15  118.401733  128.649597  123.272163
2021-01-16  117.649498  128.446579  122.900276
2021-01-17  119.322922  130.186584  123.523125
2021-01-18  117.754303  128.946533  122.647331
2021-01-19  118.098427  129.045151  123.872612
2021-01-20  117.775459  129.269470  122.622276
2021-01-21  116.197449  129.602325  123.174355
2021-01-22  117.485069  129.664856  123.424995
2021-01-23  116.223900  130.558914  123.748756
2021-01-24  115.928848  130.304184  124.099792
2021-01-25  116.006180  130.210938  123.257805
2021-01-26  116.588120  130.897644  123.646027]


Ground truth:

In [205]:
print(df_ibm_test.iloc[-(prediction_length[1]):]['Adj Close'])

Date
2021-01-07    127.289062
2021-01-08    126.835121
2021-01-11    126.884468
2021-01-12    127.506165
2021-01-13    125.246353
2021-01-14    127.269318
2021-01-15    126.696968
2021-01-19    127.318665
2021-01-20    128.364685
2021-01-21    129.913971
2021-01-22    117.045937
2021-01-25    117.016335
2021-01-26    120.874763
2021-01-27    120.855034
2021-01-28    118.496552
2021-01-29    117.539337
2021-02-01    118.950485
2021-02-02    117.864990
2021-02-03    117.549210
2021-02-04    119.424149
Name: Adj Close, dtype: float64


### Predicting Apple Stock

In [192]:
# get all input and target (test) time series
# input_ts = [df_ibm_train, df_aapl_train, df_amzn_train, df_googl_train]
# target_ts = [df_ibm_test, df_aapl_test, df_amzn_test, df_googl_test]

input_ts = [df_aapl_train]
target_ts = [df_aapl_test]

# get the prediction from the predictor
json_prediction = json_predictor.predict(input_ts)



In [193]:
print(json_prediction)

[                   0.1         0.9         0.5
2021-01-07  125.093613  132.763382  129.110001
2021-01-08  125.319176  133.149612  128.552902
2021-01-09  123.105209  133.478683  128.995026
2021-01-10  124.570038  134.422852  129.220093
2021-01-11  123.412041  134.891541  129.760132
2021-01-12  125.122711  136.418060  130.931061
2021-01-13  123.706871  135.310989  130.589737
2021-01-14  124.589447  136.117706  130.782318
2021-01-15  123.423264  137.367249  130.409927
2021-01-16  123.571732  138.207718  131.052139
2021-01-17  125.403740  139.851944  131.624786
2021-01-18  123.532669  139.180222  130.335739
2021-01-19  124.197906  138.609848  131.049896
2021-01-20  124.423309  140.104279  130.706589
2021-01-21  122.195633  140.038956  131.790817
2021-01-22  124.620186  140.600830  132.093918
2021-01-23  122.096970  141.768021  132.417282
2021-01-24  121.327774  141.081955  133.163788
2021-01-25  122.702339  141.449966  132.317459
2021-01-26  121.980942  141.936066  131.625092]


Ground truth:

In [206]:
print(df_aapl_test.iloc[-(prediction_length[1]):]['Adj Close'])

Date
2021-01-07    130.724655
2021-01-08    131.852966
2021-01-11    128.787552
2021-01-12    128.607819
2021-01-13    130.694702
2021-01-14    128.717667
2021-01-15    126.950294
2021-01-19    127.639267
2021-01-20    131.832993
2021-01-21    136.665771
2021-01-22    138.862503
2021-01-25    142.706757
2021-01-26    142.946396
2021-01-27    141.848038
2021-01-28    136.885452
2021-01-29    131.763107
2021-02-01    133.939850
2021-02-02    134.788589
2021-02-03    133.740158
2021-02-04    137.184998
Name: Adj Close, dtype: float64


### Predicting Amazon.com Stock

In [207]:
# get all input and target (test) time series
# input_ts = [df_ibm_train, df_aapl_train, df_amzn_train, df_googl_train]
# target_ts = [df_ibm_test, df_aapl_test, df_amzn_test, df_googl_test]

input_ts = [df_amzn_train]
target_ts = [df_amzn_test]

# get the prediction from the predictor
json_prediction = json_predictor.predict(input_ts)



In [208]:
print(json_prediction)

[                    0.1          0.9          0.5
2021-01-07  3094.219482  3265.937744  3186.451416
2021-01-08  3091.069092  3266.278809  3184.170898
2021-01-09  3084.891113  3272.152832  3176.452393
2021-01-10  3108.677979  3284.607910  3177.629883
2021-01-11  3093.076904  3285.515381  3173.991943
2021-01-12  3102.929443  3308.193115  3195.529541
2021-01-13  3092.083496  3309.724121  3194.941162
2021-01-14  3103.580078  3332.495117  3209.557617
2021-01-15  3094.831787  3315.429932  3202.861816
2021-01-16  3106.655029  3335.878174  3203.782471
2021-01-17  3081.158447  3334.591553  3210.318604
2021-01-18  3087.839600  3347.077637  3209.628418
2021-01-19  3084.959961  3369.511963  3210.942627
2021-01-20  3106.575195  3355.424316  3228.671631
2021-01-21  3113.171143  3378.174072  3235.158447
2021-01-22  3131.904053  3404.039795  3229.553955
2021-01-23  3125.084961  3356.024658  3236.537354
2021-01-24  3119.146484  3369.903076  3231.877686
2021-01-25  3096.519775  3349.702393  3219.257324

Ground truth:

In [211]:
print(df_amzn_test.iloc[-(prediction_length[1]):]['Adj Close'])

Date
2021-01-07    3162.159912
2021-01-08    3182.699951
2021-01-11    3114.209961
2021-01-12    3120.830078
2021-01-13    3165.889893
2021-01-14    3127.469971
2021-01-15    3104.250000
2021-01-19    3120.760010
2021-01-20    3263.379883
2021-01-21    3306.989990
2021-01-22    3292.229980
2021-01-25    3294.000000
2021-01-26    3326.129883
2021-01-27    3232.580078
2021-01-28    3237.620117
2021-01-29    3206.199951
2021-02-01    3342.879883
2021-02-02    3380.000000
2021-02-03    3312.530029
2021-02-04    3331.000000
Name: Adj Close, dtype: float64


### Predicting Alphabet Inc. Stock

In [212]:
# get all input and target (test) time series
# input_ts = [df_ibm_train, df_aapl_train, df_amzn_train, df_googl_train]
# target_ts = [df_ibm_test, df_aapl_test, df_amzn_test, df_googl_test]

input_ts = [df_googl_train]
target_ts = [df_googl_test]

# get the prediction from the predictor
json_prediction = json_predictor.predict(input_ts)



In [213]:
print(json_prediction)

[                    0.1          0.9          0.5
2021-01-07  1675.028687  1786.291382  1732.208374
2021-01-08  1668.661255  1782.718994  1733.668701
2021-01-09  1677.693115  1785.289429  1730.733521
2021-01-10  1684.309937  1793.493652  1733.876587
2021-01-11  1673.302368  1784.088989  1732.772705
2021-01-12  1666.367798  1798.504395  1736.104614
2021-01-13  1673.352051  1815.874634  1725.143311
2021-01-14  1668.871460  1817.390625  1741.362305
2021-01-15  1667.125122  1802.829712  1738.277466
2021-01-16  1668.462524  1815.500122  1737.886963
2021-01-17  1685.492798  1819.410645  1749.690918
2021-01-18  1677.694702  1810.656616  1745.091187
2021-01-19  1680.247192  1837.997070  1773.494385
2021-01-20  1692.805298  1827.459229  1767.529419
2021-01-21  1698.426025  1845.270996  1760.774292
2021-01-22  1707.861084  1840.332886  1764.793457
2021-01-23  1715.499634  1863.902222  1773.113525
2021-01-24  1719.811523  1857.586914  1782.431396
2021-01-25  1714.488770  1855.269043  1781.574951

Ground truth:

In [214]:
print(df_googl_test.iloc[-(prediction_length[1]):]['Adj Close'])

Date
2021-01-07    1774.339966
2021-01-08    1797.829956
2021-01-11    1756.290039
2021-01-12    1737.430054
2021-01-13    1747.250000
2021-01-14    1730.920044
2021-01-15    1727.619995
2021-01-19    1784.469971
2021-01-20    1880.069946
2021-01-21    1884.150024
2021-01-22    1892.560059
2021-01-25    1894.280029
2021-01-26    1907.949951
2021-01-27    1818.939941
2021-01-28    1853.199951
2021-01-29    1827.359985
2021-02-01    1893.069946
2021-02-02    1919.119995
2021-02-03    2058.879883
2021-02-04    2053.629883
Name: Adj Close, dtype: float64


### Get a Prediction

We can then use this function to get a prediction for a formatted time series!

In the next cell, I'm getting an input time series and known target, and passing the formatted input into the predictor endpoint to get a resultant prediction.

In [265]:
df_ibm_valid['Adj Close'].head()

Date
2019-12-27    126.834549
2019-12-30    124.527962
2019-12-31    125.681244
2020-01-02    126.975204
2020-01-03    125.962540
Name: Adj Close, dtype: float64

In [ ]:
# get all input and target (test) time series
input_ts = df_ibm_valid['Adj Close']
target_ts = time_series

# get formatted input time series
json_input_ts = json_predictor_input(input_ts)

# get the prediction from the predictor
json_prediction = predictor.predict(json_input_ts)

#print(json_prediction)

## Delete the Endpoint

Try your code out on different time series. You may want to tweak your DeepAR hyperparameters and see if you can improve the performance of this predictor.

When you're done with evaluating the predictor (any predictor), make sure to delete the endpoint.

In [ ]:
## TODO: delete the endpoint
predictor.delete_endpoint()